In [ ]:
import sys
import os

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import scipy
import pandas as pd
import datetime
import copy

import matplotlib.dates as mdates
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
from matplotlib import ticker as mticker


# Local files
__root__ = '../../'
__pydirs__ = ['py', 'settings']
module_path = os.path.abspath(os.path.join(__root__[:-1]))
if module_path not in sys.path:
    for pydir in __pydirs__:
        sys.path.append(f"{module_path}/{pydir}/")

import T23_setup

import log_ret

import misc_functions as misc_fns
import plot_functions as plot_fns
import dist_functions as dist_fns
import hawkes_functions as hwks_fns
import log_ret_functions as lr_fns
import cvol_functions as cvol_fns
import greek_roman as GR

In [ ]:
# Set seed
np.random.seed(3791)

In [ ]:
plot_fns.plt_setup(plt, SMALL_SIZE=6.5, MEDIUM_SIZE=6.5, BIGGER_SIZE=8)
cols = plot_fns.calc_cols()
cols_CI = plot_fns.calc_cols_CI()
cols_tail = plot_fns.calc_cols_tail(rows=True)

In [ ]:
r_r = f"r"

resid_r = f"\\tilde{{{r_r}}}"
sim_r = f"\\acute{{{r_r}}}"

tilde_t_cont = "\\Lambda"
tilde_t_disc = "\\Pi"

In [ ]:
Tmisc = T23_setup.T23_misc()
fstmp = T23_setup.T23_Cfstmp(root=__root__, n=4, title='T23')
fstmp.calc_fnsubs_default()

In [ ]:
fix_a_lambda = True
tick='SPX'
_setup_T23 = T23_setup.setup_T23(__C__='Cm4', __fit__=False, __vares__=False,
                                    set_cv=False, set_tt=True,
                                    tt_fix_a_lambda=fix_a_lambda,
                                    all_end=pd.Timestamp('2023-09-16'),
                                    )

In [ ]:
_setup_T23['tt_a_u']

In [ ]:
B20 = log_ret.log_ret(dir_base=__root__, ticks=_setup_T23['ticks'])
B20.set_tk(**misc_fns.dict_key_rm(_setup_T23, keys_rm='ticks'))

In [ ]:
cv_mp_tick_time = B20.set_cv_mp_tick(set_am_stats=True, set_eval=True, append_fit_cv_w=True, append_vrs_cv_w=False, write_tick=False)

In [ ]:
B20.set_tt_hb_mp_tick(write_tick=False, 
                        a_u_rng=[1], tt_fix_a_lambda=fix_a_lambda, tt_read_params=True, tt_hb_read_params=True, tt_fit_if_need=False, append_fit_tt_w=False, set_eval=True, append_fit_tt_hb_w=True, append_vrs_tt_hb_w=False)

In [ ]:

tails = [f"L", f"R"]
signs = np.array([-1,1])


T_year = 252


str_dp=1
str_exp_plus=True
str_exp_zeros=2


tail_fit_setup = {
                    'u': np.outer([-1,1], [0.005, 0.01, 0.025, 0.05, 0.075, 0.1, 0.2]),
                    'a_u': np.outer([1,1], _setup_T23['tt_a_u']),
                }




for tick in B20.tk:
    B20.tk[tick]._T = {period: B20.tk[tick].pd.lr["p"][period].values.size for period in ['train', 'all']}
    B20.tk[tick]._N = {period: np.array([[np.sum(signs[i]*B20.tk[tick].pd.lr["p"][period].values > signs[i]*tail_fit_setup['u'][i,j]) for j in range(tail_fit_setup['u'].shape[1])] for i in range(signs.size)]) for period in ['train', 'all']}
    for val in ['T', 'N']:
        B20.tk[tick].__dict__[f'_{val}']['forecast'] = B20.tk[tick].__dict__[f'_{val}']['all'] - B20.tk[tick].__dict__[f'_{val}']['train']



    B20.tk[tick]._stat_dist_fit = dict()
    for dist in ['norm', 't']:
        B20.tk[tick]._stat_dist_fit[f"{dist}_ML"] = dist_fns.calc_scipy_dist_est_se_log_L(data=B20.tk[tick].pd.lr["p"]["train"].values, dist=dist)
    Q_rng = [0.25, 0.75]
    B20.tk[tick]._stat_dist_fit[f"norm_Q"] = dist_fns.calc_scipy_dist_est_se_log_L(data=B20.tk[tick].pd.lr["p"]["train"].values, dist='norm', est=np.array([np.median(B20.tk[tick].pd.lr["p"]["train"].values),
                                                                                                                                        (np.diff(np.quantile(B20.tk[tick].pd.lr["p"]["train"].values, Q_rng))/np.diff(scipy.stats.norm().ppf(Q_rng)))[0]]))
    
    meanstd = B20.tk[tick]._stat_dist_fit[f"norm_ML"]['est']
    B20.tk[tick]._stat_dist_fit["t_ML_f"] = dist_fns.calc_scipy_dist_est_se_log_L(data=B20.tk[tick].pd.lr["p"]["train"].values, dist='t', est=np.array([meanstd[0], meanstd[1],
                                                                                                                                                scipy.stats.t.fit((B20.tk[tick].pd.lr["p"]["train"].values-meanstd[0])/meanstd[1], floc=0, fscale=1)[0]]))
    
    B20.tk[tick]._stat_dist = dict()
    B20.tk[tick]._stat_dist_symb = dict()
    B20.tk[tick]._stat_dist_symb_fit = dict()
    B20.tk[tick]._stat_dist_symb_mod = dict()
    for dist in B20.tk[tick]._stat_dist_fit:
        B20.tk[tick]._stat_dist[dist] = scipy.stats.__getattribute__(dist[:dist.find("_")])(*B20.tk[tick]._stat_dist_fit[dist]['est'])
        B20.tk[tick]._stat_dist_symb[dist] = dist_fns.get_scipy_dist_symb(dist[:dist.find('_')])
        B20.tk[tick]._stat_dist_symb_fit[dist] = dist_fns.get_scipy_dist_symb(dist[:dist.find('_')])
        B20.tk[tick]._stat_dist_symb_mod[dist] = dist_fns.get_scipy_dist_symb(dist[:dist.find('_')])

    for dist in ['norm', 't']:
        #B20.tk[tick]._stat_dist_symb_fit[f"{dist}_ML"] = B20.tk[tick]._stat_dist_symb_fit[f"{dist}_ML"] + f"_{{\\mathrm{{ML}}}}"
        B20.tk[tick]._stat_dist_symb_fit[f"{dist}_ML"] = B20.tk[tick]._stat_dist_symb_fit[f"{dist}_ML"] + f"_{{W}}"
        B20.tk[tick]._stat_dist_symb_mod[f"{dist}_ML"] = f"{cvol_fns.calc_cv_vol_symb(val='uv')}^{{{B20.tk[tick]._stat_dist_symb[f'{dist}_ML']}}}"
    
    B20.tk[tick]._stat_dist_symb_fit[f"norm_Q"] = B20.tk[tick]._stat_dist_symb_fit[f"norm_Q"] + f"_{{Q}}"
    #B20.tk[tick]._stat_dist_symb_fit[f"t_ML_f"] = B20.tk[tick]._stat_dist_symb_fit[f"t_ML_f"] + f"_{{\\mathrm{{ML}}}}^{{f}}"
    B20.tk[tick]._stat_dist_symb_fit[f"t_ML_f"] = B20.tk[tick]._stat_dist_symb_fit[f"t_ML_f"] + f"_{{W}}^{{f}}"

    for mod in [f"norm_Q", f"t_ML_f"]:
        B20.tk[tick]._stat_dist_symb_mod[mod] = f"{cvol_fns.calc_cv_vol_symb(val='uv')}^{{{B20.tk[tick]._stat_dist_symb_fit[mod]}}}"

        

    B20.tk[tick]._tail_fit = {key: dist_fns.calc_tail_fit(B20.tk[tick].pd.lr["p"]["train"].values, **{key: tail_fit_setup[key]}) for key in tail_fit_setup}


    for key in B20.tk[tick]._tail_fit:
        B20.tk[tick]._tail_fit[key]['lambda']['inv_lambda_str'] = np.array([[misc_fns.scif_string((1/T_year)/B20.tk[tick]._tail_fit[key]['lambda']['a_lambda'][i,j], dp=str_dp, exp_plus=str_exp_plus, exp_zeros=str_exp_zeros) 
                                                                                for j in range(B20.tk[tick]._tail_fit[key]['lambda']['a_lambda'].shape[1])] for i in range(B20.tk[tick]._tail_fit[key]['lambda']['a_lambda'].shape[0])], dtype=object)
        
        B20.tk[tick]._tail_fit[key]['lambda']['inv_lambda_str'][np.isinf(B20.tk[tick]._tail_fit[key]['lambda']['a_lambda'])] = f"$>{misc_fns.scif_string(B20.tk[tick]._T['train']/T_year, dp=str_dp, exp_plus=str_exp_plus, exp_zeros=str_exp_zeros, app_dlrs=False)}$"


    B20.tk[tick]._inv_lambda = dict()
    B20.tk[tick]._inv_lambda["N"] = {period: (B20.tk[tick]._T[period]/T_year)/B20.tk[tick]._N[period] for period in B20.tk[tick]._T}
    
    for dist in B20.tk[tick]._stat_dist:
        #B20.tk[tick]._inv_lambda[dist] = np.array([(1/T_year)/(i + ((-1)**i)*B20.tk[tick]._stat_dist[dist].cdf(tail_fit_setup['u'][i,:])) for i in range(signs.size)])
        B20.tk[tick]._inv_lambda[dist] = np.array([np.exp(-B20.tk[tick]._stat_dist[dist].logcdf(i*2*B20.tk[tick]._stat_dist[dist].mean() + ((-1)**i)*tail_fit_setup['u'][i,:])) for i in range(tail_fit_setup['u'].shape[0])])/T_year
        
        

    B20.tk[tick]._inv_lambda_str = dict()
    for dist in  B20.tk[tick]._stat_dist:
        B20.tk[tick]._inv_lambda_str[dist] = np.array([[misc_fns.scif_string(B20.tk[tick]._inv_lambda[dist][i,j], dp=str_dp, exp_plus=str_exp_plus, exp_zeros=str_exp_zeros) for j in range(tail_fit_setup['u'].shape[1])] for i in range(signs.size)], dtype=object)

    B20.tk[tick]._inv_lambda_str["N"] = {period: np.array([[misc_fns.scif_string(B20.tk[tick]._inv_lambda["N"][period][i,j], dp=str_dp, exp_plus=str_exp_plus, exp_zeros=str_exp_zeros) for j in range(tail_fit_setup['u'].shape[1])] for i in range(signs.size)], dtype=object) for period in B20.tk[tick]._inv_lambda["N"]}
    for period in B20.tk[tick]._inv_lambda_str["N"]:
        B20.tk[tick]._inv_lambda_str["N"][period][np.isinf(B20.tk[tick]._inv_lambda["N"][period])] = f"$>{misc_fns.scif_string(B20.tk[tick]._T[period]/T_year, dp=str_dp, exp_plus=str_exp_plus, exp_zeros=str_exp_zeros, app_dlrs=False)}$"
        B20.tk[tick]._inv_lambda_str["N"][period][np.isnan(B20.tk[tick]._inv_lambda["N"][period])] = f""

In [ ]:

bounds = ['start', 'end', 'T']
periods = ['all', 'train', 'forecast']

C_str = fstmp.calc_C_str()




dates_ts_latex = {tick: {period: {bound: None for bound in bounds} for period in periods} for tick in B20.tk}
dates_T_latex = {tick: {period: None for period in periods} for tick in B20.tk} 
T = {period: None for period in periods}


for tick in B20.tk:
    for period in periods:
        if period=='forecast':
            dates_ts_latex[tick][period]['start'] = B20.tk[tick].pd._dates['train'][-1].strftime('%Y-%m-%d')
            dates_ts_latex[tick][period]['end'] = B20.tk[tick].pd._dates['all'][-1].strftime('%Y-%m-%d')
            T[period] = T['all'] - T['train']
        else:
            for i in range(2):
                bound = bounds[i]
                dates_ts_latex[tick][period][bound] = B20.tk[tick].pd._dates[period][i].strftime('%Y-%m-%d')
            T[period] = B20.tk[tick].pd.lr['p'][period].values.size

        dates_ts_latex[tick][period]['T'] = Tmisc.capp_tex_num(T[period])


multi_all_start = True
print(f"% Dates")
if multi_all_start:
    print(f"\\newcommand{{\\{C_str['l']}rsDateAllStartALL}}{{}}")
else:
    print(f"\\newcommand{{\\{C_str['l']}rsDateAllStartALL}}{{{_setup_T23['all_start'].strftime('%Y-%m-%d')}}}")

print(f"\\newcommand{{\\{C_str['l']}rsDateAllEndALL}}{{{_setup_T23['all_end'].strftime('%Y-%m-%d')}}}")
if multi_all_start:
    print(f"\\newcommand{{\\{C_str['l']}rsDateTrainStartALL}}{{}}")
else:
    print(f"\\newcommand{{\\{C_str['l']}rsDateTrainStartALL}}{{{_setup_T23['all_start'].strftime('%Y-%m-%d')}}}")
print(f"\\newcommand{{\\{C_str['l']}rsDateTrainEndALL}}{{{_setup_T23['train_end'].strftime('%Y-%m-%d')}}}")
print(f"\\newcommand{{\\{C_str['l']}rsDateForecastStartALL}}{{{_setup_T23['train_end'].strftime('%Y-%m-%d')}}}")
print(f"\\newcommand{{\\{C_str['l']}rsDateForecastEndALL}}{{{_setup_T23['all_end'].strftime('%Y-%m-%d')}}}")
print('')

for period in periods:
    for bound in bounds:
        for tick in B20.tk:
            print(f"\\newcommand{{\\{C_str['l']}rsDate{period.capitalize()}{bound.capitalize()}{tick}}}{{{dates_ts_latex[tick][period][bound]}}}")
        print('')


In [ ]:
tick='SPX'
mode='bi'
mark='mark'

B20.tk[tick].tt.tt[mode][mark]._parameters.print()

In [ ]:
B20.tk[tick].tt.tt[mode][mark]._parameters.a_u.get_value()

In [ ]:
B20.tk[tick].tt.tt[mode][mark]._parameters.a_lambda.get_value()

In [ ]:
B20.tk[tick].tt.tt[mode][mark]._parameters.a_lambda.get_default()

In [ ]:
plt.rcParams['figure.dpi'] = 300

In [ ]:
plt.rcParams['figure.dpi']

In [ ]:
ms = 12
markers = ['v', '^']

xlim_abs = 4.25
xlim = xlim_abs*np.array([-1,1])
ylim = [-0.25, 0.15]


ticks = np.transpose([
                ["SPX", "DAX", "NKX"],
                ["DJI", "CAC", "HSI"],
            ])

I_max = len(ticks)
J_max = len(ticks[0])

fig_norm_qq = plot_fns.rc_plot_setup(name=f'TICK_r_qq', nrows=I_max, ncols=J_max)

for i in range(I_max):
    for j in range(J_max):

        tick = ticks[i][j]

        data = copy.deepcopy(B20.tk[tick].pd.lr['p']['train'])
        data_2 = copy.deepcopy(B20.tk[tick].pd.lr['p']['forecast'][0][0])

        normal_mean = scipy.stats.norm(*scipy.stats.norm.fit(data))

        fig_norm_qq_quantiles = (0.5+np.array(range(data.values.size)))/data.values.size
        fig_norm_qq_x = scipy.stats.norm().ppf(fig_norm_qq_quantiles)
        fig_norm_qq_x_ls = np.linspace(fig_norm_qq_x[0], fig_norm_qq_x[-1], 201)

        fig_norm_qq_quantiles_2 = (0.5+np.array(range(data_2.values.size)))/data_2.values.size
        fig_norm_qq_x_2 = scipy.stats.norm().ppf(fig_norm_qq_quantiles_2)

        fig_norm_qq["ax"][i][j].scatter(fig_norm_qq_x, np.sort(data.values), color=cols[0], lw=0, s=ms, marker=markers[0])
        fig_norm_qq["ax"][i][j].scatter(fig_norm_qq_x_2, np.sort(data_2.values), color=cols[7], lw=0, s=ms, marker=markers[1])


        #fig_norm_qq["ax"][i][j].plot(fig_norm_qq_x[[0,-1]], normal_mean.ppf(scipy.stats.norm().cdf(fig_norm_qq_x[[0,-1]])), color=cols[9], linewidth=1.5)
        #fig_norm_qq["ax"][i][j].plot(fig_norm_qq_x[[0,-1]], normal_median.ppf(scipy.stats.norm().cdf(fig_norm_qq_x[[0,-1]])), color="orange", linewidth=1.5) 

        fig_norm_qq["ax"][i][j].plot(xlim, normal_mean.ppf(scipy.stats.norm().cdf(xlim)), color=cols[9], linewidth=1.5, zorder=0)


        #fig_norm_qq["ax"][i][j].set_ylabel(f"{tick} $r_t$")
        

        #fig_norm_qq["ax"][i][j].set_ylim(0.25*np.array([-1,1]))

        a_u = B20.tk[tick].tt.tt["bi"]["mark"]._parameters.a_u.get_value()
        u_q = np.array(range(2)) + np.array([1,-1])*a_u
        if True:
            u_y = B20.tk[tick].tt.tt["bi"]["mark"]._parameters.u.get_value()
            u_x = (B20.tk[tick].tt.tt["bi"]["mark"]._parameters.u.get_value() - normal_mean.mean())/normal_mean.std()
        else:
            u_y = normal_mean.ppf(u_q)
            u_x = scipy.stats.norm().ppf(u_q)
        if False:
            for p in range(2):
                fig_norm_qq["ax"][i][j].plot(np.repeat(u_x[p], 2), np.array([u_y[p], ((-1)**(1+p))*5]), linewidth=0.5, linestyle="--", color="black")
                fig_norm_qq["ax"][i][j].plot(np.array([u_x[p], ((-1)**(1+p))*5]), np.repeat(u_y[p], 2), linewidth=0.5, linestyle="--", color="black")

        xy_box = hwks_fns.annotate_model_Hawkes(ax=fig_norm_qq["ax"][i][j], 
                                                    #tick=tick, series="r_{{t}}", plot_tick=True, 
                                                    str_tick=tick, plot_tick=True, 
                                                    an_h='left', an_v='top', 
                                                    an_x0=0.015, an_y0=0.03, flip_x0ry0t=True, an_mode=fstmp.get_an_mode())

        
        
        fig_norm_qq["ax"][i][j].tick_params(labeltop=False, labelright=False)
        if j==0:
            fig_norm_qq["ax"][i][j].set_ylabel(f"$r_t$")
        
        if i==I_max-1:
            fig_norm_qq["ax"][i][j].set_xlabel(f"Normal quantiles")


        fig_norm_qq["ax"][i][j].set_xlim(xlim)
        fig_norm_qq["ax"][i][j].set_ylim(ylim)


    
i=0
j=0
fig_norm_qq["ax"][i][j].legend([fig_norm_qq["ax"][i][j].get_children()[p] for p in range(3)], ["In-sample", "Out-of-sample", f"$\\hat{{{dist_fns.get_scipy_dist_symb(dist='norm')}}}_{{W}}$", 
                                                                                                       #f"$u^{{\\leftrightharpoons}}{{\\left(a_u = {a_u[0]:.3f}\\right)}}$"
                                                                                                       ], loc="lower right")
    

fig_norm_qq["fig"].set_size_inches(5.7, 7)
fig_norm_qq["fig"].tight_layout()

fstmp.fig_export(fig_norm_qq, height=7, formats=["pdf", "png"], dpi=fstmp.get_dpi())


In [ ]:
vol = 'GARCH'

p = 1
o = 0
q = 1

resid_symb = f'\\tilde{{r}}'


dists = ["norm", "t"]


K = len(dists)
#dists_cols = [cols[3], cols[1], cols[5]]

mods_cols = [cols[9], cols[0]]

mods_select = np.arange(K)

mods_markers = ['v', '^']

ms = 9
marker = 'D'

xlim_abs = 4.25
ylim_abs = 10.5

xlim = xlim_abs*np.array([-1,1])
#ylim = ylim_abs*np.array([-1,1])
ylim = np.array([-13.5, 7.5])

xs = np.linspace(-xlim_abs,xlim_abs,2)

CI = [0.95, 0.99]
N_CI  = len(CI)
cols_CI = plot_fns.calc_cols_CI(N_CI=N_CI)


ticks = np.transpose([
                ["SPX", "DAX", "NKX"],
                ["DJI", "CAC", "HSI"],
            ])

I_max = np.shape(ticks)[0]
J_max = np.shape(ticks)[1]


fig_norm_qq = plot_fns.rc_plot_setup(name=f'TICK_tilde_r_qq_cv', nrows=I_max, ncols=J_max)

for i in range(I_max):
    for j in range(J_max):
        tick = ticks[i][j]

        mods = [B20.tk[tick].cv.cv.cv_v[vol].cv_p[p][o][q].cv_d[dist] for dist in dists]


        data = copy.deepcopy(B20.tk[tick].pd.lr['p']['all'])
        

        period_t = B20.tk[tick].pd.get_periods_t_offset()['train']

        

        epsilon = [None for k in range(K)]
        tilde_r = [None for k in range(K)]
        factor = np.ones(K, dtype=int)


        
        T = np.size(data[period_t[0]:period_t[1]])
        fig_norm_qq_quantiles = (0.5+np.array(range(T)))/T
        fig_norm_qq_x = scipy.stats.norm().ppf(fig_norm_qq_quantiles)
        fig_norm_qq_x_ls = np.linspace(fig_norm_qq_x[0], fig_norm_qq_x[-1], 201)

        CI_y = np.linspace(ylim[0], ylim[1], 201)

        CI_x = scipy.stats.norm.ppf(np.array([[scipy.stats.norm().cdf(CI_y) + ((-1)**(1+k))*np.sqrt(np.log(2/((1-CI[j])/2)))/np.sqrt(2*T) for j in range(len(CI))] for k in range(2)]))

        
        #fig_norm_qq["ax"][i][j].scatter(fig_norm_qq_x, np.sort(data.values), color=cols[7], marker=marker, s=ms, lw=0)

        #fig_norm_qq["ax"][i][j].plot(fig_norm_qq_x[[0,-1]], normal_mean.ppf(scipy.stats.norm().cdf(fig_norm_qq_x[[0,-1]])), color='red', linewidth=1.5)
        #fig_norm_qq["ax"][i][j].plot(fig_norm_qq_x[[0,-1]], normal_median.ppf(scipy.stats.norm().cdf(fig_norm_qq_x[[0,-1]])), color="orange", linewidth=1.5) 
        if False:
            for i in range(N_CI):
                j = N_CI-(1+i)
                fig_norm_qq["ax"][i][j].fill_betweenx(y=CI_y, x1=CI_x[0][j], x2=CI_x[1][j], color=cols_CI[j])

        
        for m in mods_select:
            if Tmisc.get_sd_not_scale() and mods[m]._dist=='t':
                nu = mods[m]._bdist.kwds['df']
                factor[m] = np.sqrt(nu/(nu-2))

            epsilon[m] = np.sort(((data - mods[m]._bdist.kwds['loc'])/mods[m]._bdist.kwds['scale'])[period_t[0]:period_t[1]])
            tilde_r[m] = np.sort(scipy.stats.norm().ppf(mods[m].F_X[period_t[0]:period_t[1]]))

            

            fig_norm_qq["ax"][i][j].scatter(fig_norm_qq_x, tilde_r[m], color=mods_cols[m], marker=mods_markers[m], s=ms, lw=0)

            #fig_norm_qq["ax"][i][j].plot(fig_norm_qq_x_ls, B20.tk[tick]._stat_dist[dists[k]].ppf(scipy.stats.norm().cdf(fig_norm_qq_x_ls)), color=dists_cols[k], linewidth=1.5)

        
        #fig_norm_qq["ax"][i][j].plot(fig_norm_qq_x_ls, B20.tk[tick]._stat_dist['t_ML_f'].ppf(scipy.stats.norm().cdf(fig_norm_qq_x_ls)), color=cols[5], linewidth=1.5)
        fig_norm_qq["ax"][i][j].plot(xs, xs, color=cols[7], lw=2, zorder=-1)

        #fig_norm_qq["ax"][i][j].set_ylabel(f"{tick} $r_t$")
        


        fig_norm_qq["ax"][i][j].set_xlim(xlim)
        fig_norm_qq["ax"][i][j].set_ylim(ylim)
        #fig_norm_qq["ax"][i][j].set_ylim(0.25*np.array([-1,1]))

        if False:     
            u_q = np.array(range(2)) + np.array([1,-1])*B20.tk[tick].tt.tt["bi"]["mark"]._parameters.a_lambda.get_value()
            if True:
                u_y = B20.tk[tick].tt.tt["bi"]["mark"]._parameters.u.get_value()
                u_x = (B20.tk[tick].tt.tt["bi"]["mark"]._parameters.u.get_value() - normal_mean.mean())/normal_mean.std()
            else:
                u_y = normal_mean.ppf(u_q)
                u_x = scipy.stats.norm().ppf(u_q)
            for p in range(2):
                fig_norm_qq["ax"][i][j].plot(np.repeat(u_x[p], 2), np.array([u_y[p], ((-1)**(1+p))*5]), linewidth=0.5, linestyle="--", color="black")
                fig_norm_qq["ax"][i][j].plot(np.array([u_x[p], ((-1)**(1+p))*5]), np.repeat(u_y[p], 2), linewidth=0.5, linestyle="--", color="black")





        xy_box = hwks_fns.annotate_model_Hawkes(ax=fig_norm_qq["ax"][i][j], 
                                                #tick=tick, series="r_{{t}}", plot_tick=True, 
                                                str_tick=tick, plot_tick=True, 
                                                an_h='left', an_v='top', 
                                                an_x0=0.015, an_y0=0.03, flip_x0ry0t=True, an_mode=fstmp.get_an_mode())

        if i==0 and j==0:
                fig_norm_qq["ax"][i][j].legend([fig_norm_qq["ax"][i][j].get_children()[p] for p in np.arange(3)], 
                                   np.concatenate([[f"$\\hat{{{resid_symb}}}_{{t}} | {cvol_fns.calc_cv_symb(vol=vol, o=mods[m]._o, dist=mods[m]._dist)}$" for m in mods_select], [f"$\\mathcal{{N}}_{{\\left(0,1\\right)}}$"]]), loc="lower right",
                                   bbox_to_anchor=plot_fns.calc_bbox(xy_boxes=[xy_box], along_x=False, along_y=True)
                                   )
        

        fig_norm_qq['ax'][i][j].tick_params(labelleft=(j==0), labelright=False, labelbottom=(i==I_max-1), labeltop=False)
        if i==I_max-1:
            fig_norm_qq["ax"][i][j].set_xlabel(f"Normal quantiles")
        if j==0:
            fig_norm_qq["ax"][i][j].set_ylabel(f"${resid_symb}_{{t}}$")

        
        


fig_norm_qq["fig"].set_size_inches(5.7, 7)
fig_norm_qq["fig"].tight_layout()

fstmp.fig_export(fig_norm_qq, height=7, formats=['pdf', 'png'], dpi=fstmp.get_dpi())

